In [1]:
import torch
import cv2
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from PIL import Image
import numpy as np
from pprint import pprint
from demo import detect_cv2, get_args
import os

/home/imu/anaconda3/envs/yolo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_folder = "test"
output_folder = "output"
cfgfile = "./cfg/yolov4-tiny.cfg"
weightfile = './weights/yolov4-tiny_best.weights'
preds = detect_cv2(cfgfile, weightfile, None,
                       input_dir=input_folder, out_dir=output_folder)

Loading weights from ./weights/yolov4-tiny_best.weights... Done!
test/54.jpg
[0.45519906, -0.027147532, 0.833853, 0.6126584, 0.9197522, 0.9197522, 5]
Bus: 0.919752
save plot results to output/54.jpg
test/22.jpg
[0.77746135, 0.3731308, 1.0011342, 0.81736517, 0.8643382, 0.8643382, 2]
FourWheeler: 0.864338
[0.6226889, 0.13857408, 0.87049526, 0.37831682, 0.6134954, 0.6134954, 2]
FourWheeler: 0.613495
save plot results to output/22.jpg
test/62.jpg
save plot results to output/62.jpg
test/28.jpg
[0.448102, -0.0072150826, 0.8382766, 0.5860722, 0.9757485, 0.9757485, 5]
Bus: 0.975748
save plot results to output/28.jpg
test/48.jpg
[0.17392015, 0.15646051, 0.48536468, 0.4778692, 0.9901653, 0.9901653, 2]
FourWheeler: 0.990165
[0.6858729, 0.40250388, 0.9792942, 0.9525857, 0.9746385, 0.9746385, 2]
FourWheeler: 0.974639
save plot results to output/48.jpg
test/81.jpg
[0.38587445, 0.045107648, 0.6695777, 0.37736642, 0.9924034, 0.9924034, 2]
FourWheeler: 0.992403
[0.6858076, 0.3933977, 0.9786018, 0.96307

In [3]:
noise_original = Image.open('light-noise-alpha.png')
start = 1
finish = 6
step = 0.2

In [4]:
input_folder = "test"
output_folder = "output"

folder_list = []

for key in preds.keys():
    print(f"Image - {key}...")
    if len(preds[key]["bboxes"]) > 0:
        try:
            os.mkdir(f"exp_script/output/{key}")
        except Exception as e:
            print(e)

        img = np.array(Image.open(f"{input_folder}/{key}.jpg"))

        width = img.shape[1]
        height = img.shape[0]

        rgb = (255, 0, 0)
        bbox_thick = int(0.6 * (height + width) / 600)

        with open(f"{input_folder}/{key}.txt", "r", encoding="utf-8") as txt_file:
            bbox_data = txt_file.read().strip()
            bbox_data = bbox_data.split("\n")
            _labels = []
            _bbox = []
            for row in bbox_data:
                _row = row.split(" ")
                x_center, y_center, b_width, b_height = float(_row[1])*width, float(_row[2])*height, \
                    float(_row[3])*width, float(_row[4])*height

                x1, y1, x2, y2 = int(x_center-(b_width/2)), int(y_center-(b_height/2)),\
                    int(x_center+(b_width/2)), int(y_center+(b_height/2))
                
                x1 = 0 if x1 < 0 else x1
                y1 = 0 if y1 < 0 else y1
                x2 = 0 if x2 < 0 else x2
                y2 = 0 if y2 < 0 else y2

                x1 = height if x1 > height else x1
                y1 = height if y1 > height else y1
                x2 = height if x2 > height else x2
                y2 = height if y2 > height else y2

                if [x1, y1, x2, y2] not in _bbox:
                    _bbox.append([x1, y1, x2, y2])
                    _labels.append(int(_row[0]))

            #     img = cv2.rectangle(img, (x1, y1), (x2, y2), rgb, bbox_thick)

            # cv2.imwrite(f"output1/{key}.png", img)
            # img = np.array(Image.open(f"{input_folder}/{key}.jpg"))
            # rgb = (255, 255, 0)
            # for bbox in preds[key]['bboxes']:
            #     img = cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), rgb, bbox_thick)
            # cv2.imwrite(f"output2/{key}.png", img)
            preds[key]["original"] = {
                "bboxes": _bbox,
                "labels": _labels
            }
        
        for noise_ratio in np.arange(start, finish, step):
            noise_ratio = round(noise_ratio, 2)
            im_original = np.array(Image.open(f"{input_folder}/{key}.jpg"))

            for bbox in preds[key]['bboxes']:
                preds[key]['obj'] = len(preds[key]['bboxes'])
                x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]

                cropped = im_original[y1:y2, x1:x2, :]
            
                cropped_im = Image.fromarray(cropped)
                # display(cropped_im)
                print(noise_ratio)
                ny, nx = int((y2-y1)/noise_ratio), int((x2-x1)/noise_ratio)

                noise_new = noise_original.resize((nx, ny))
                
                # display(noise_new)
                noise_position = (((x2-x1)-nx)//2, ((y2-y1)-ny)//2)
                # print(x2-x1, y2-y1)
                # print(nx, ny)
                # print(noise_position)
                cropped_im.paste(noise_new, noise_position, noise_new)
                cropped_im = np.array(cropped_im)

                im_original[y1:y2, x1:x2, :] = cropped_im

                im_original = Image.fromarray(im_original)
                # display(im_original)
                file_name = str(noise_ratio).replace(".", "-")
                im_original.save(f"exp_script/output/{key}/{file_name}.jpg")
                im_original = np.array(im_original)
                
            # break
        # break


Image - 54...
[Errno 17] File exists: 'exp_script/output/54'
1.0
1.2
1.4
1.6
1.8
2.0
2.2
2.4
2.6
2.8
3.0
3.2
3.4
3.6
3.8
4.0
4.2
4.4
4.6
4.8
5.0
5.2
5.4
5.6
5.8
Image - 22...
[Errno 17] File exists: 'exp_script/output/22'
1.0
1.0
1.2
1.2
1.4
1.4
1.6
1.6
1.8
1.8
2.0
2.0
2.2
2.2
2.4
2.4
2.6
2.6
2.8
2.8
3.0
3.0
3.2
3.2
3.4
3.4
3.6
3.6
3.8
3.8
4.0
4.0
4.2
4.2
4.4
4.4
4.6
4.6
4.8
4.8
5.0
5.0
5.2
5.2
5.4
5.4
5.6
5.6
5.8
5.8
Image - 62...
Image - 28...
[Errno 17] File exists: 'exp_script/output/28'
1.0
1.2
1.4
1.6
1.8
2.0
2.2
2.4
2.6
2.8
3.0
3.2
3.4
3.6
3.8
4.0
4.2
4.4
4.6
4.8
5.0
5.2
5.4
5.6
5.8
Image - 48...
[Errno 17] File exists: 'exp_script/output/48'
1.0
1.0
1.2
1.2
1.4
1.4
1.6
1.6
1.8
1.8
2.0
2.0
2.2
2.2
2.4
2.4
2.6
2.6
2.8
2.8
3.0
3.0
3.2
3.2
3.4
3.4
3.6
3.6
3.8
3.8
4.0
4.0
4.2
4.2
4.4
4.4
4.6
4.6
4.8
4.8
5.0
5.0
5.2
5.2
5.4
5.4
5.6
5.6
5.8
5.8
Image - 81...
[Errno 17] File exists: 'exp_script/output/81'
1.0
1.0
1.2
1.2
1.4
1.4
1.6
1.6
1.8
1.8
2.0
2.0
2.2
2.2
2.4
2.4
2.6
2.6
2.8
2.8
3

In [5]:
eval_data = preds.copy()

In [6]:
input_folder = "exp_script/output/"
output_folder = "exp_script/results/"

for key in eval_data.keys():
    print(f"Image - {key}...")
    if len(eval_data[key]["bboxes"]) > 0:
        try:
            os.mkdir(f"exp_script/results/{key}")
        except Exception as e:
            print(e)

        preds_eval = detect_cv2(cfgfile, weightfile, None,
                        input_dir=f"{input_folder}{key}", out_dir=f"{output_folder}{key}")

        for key2 in preds_eval.keys():
            eval_data[key][key2] = preds_eval[key2]
# 

Image - 54...
[Errno 17] File exists: 'exp_script/results/54'
Loading weights from ./weights/yolov4-tiny_best.weights... Done!
exp_script/output/54/5-2.jpg
[0.42185888, 0.045355063, 0.53507787, 0.1429184, 0.7016381, 0.7016381, 2]
FourWheeler: 0.701638
[0.46306762, 0.050800543, 0.5658013, 0.1437541, 0.46799234, 0.46799234, 2]
FourWheeler: 0.467992
[0.45832518, -0.015598983, 0.8131453, 0.6025386, 0.9660328, 0.9660328, 5]
Bus: 0.966033
save plot results to exp_script/results/54/5-2.jpg
exp_script/output/54/2-2.jpg
[0.4214074, 0.04505757, 0.5360681, 0.14331271, 0.6809679, 0.6809679, 2]
FourWheeler: 0.680968
[0.46241766, 0.05083756, 0.5659271, 0.14631003, 0.49994022, 0.49994022, 2]
FourWheeler: 0.499940
[0.4738341, -0.001543194, 0.81446636, 0.5603883, 0.6002405, 0.6002405, 4]
HeavyCV: 0.600241
save plot results to exp_script/results/54/2-2.jpg
exp_script/output/54/3-6.jpg
[0.42173445, 0.04523203, 0.5354508, 0.14300235, 0.69558954, 0.69558954, 2]
FourWheeler: 0.695590
[0.46293435, 0.05074152

In [7]:
for noise_ratio in np.arange(start, finish, step):
    noise_ratio = round(noise_ratio, 2)
    key2 = str(noise_ratio).replace(".", "-")
    print("key2 - ", key2)
    without_noise = []
    originals = []
    preds_final = []

    for key in eval_data.keys():
        if key2 in eval_data[key].keys():
            without_noise.append(
                {
                    "boxes": torch.tensor(eval_data[key]['bboxes']),
                    "labels": torch.tensor(eval_data[key]['labels']),
                    "scores": torch.tensor(eval_data[key]['scores'])
                }.copy()
            )
            
            originals.append(
                {
                    "boxes": torch.tensor(eval_data[key]['original']['bboxes']),
                    "labels": torch.tensor(eval_data[key]['original']['labels'])
                }.copy()
            )

            preds_final.append(
                {
                    "boxes": torch.tensor(eval_data[key][key2]['bboxes']),
                    "labels": torch.tensor(eval_data[key][key2]['labels']),
                    "scores": torch.tensor(eval_data[key][key2]['scores'])
                }.copy()
            )

    metric = MeanAveragePrecision()
    metric.update(preds_final, originals)
    pprint(metric.compute())

print("original map --------------")
metric = MeanAveragePrecision()
metric.update(without_noise, originals)
pprint(metric.compute())

key2 -  1-0
{'map': tensor(0.0668),
 'map_50': tensor(0.1300),
 'map_75': tensor(0.0538),
 'map_large': tensor(0.0459),
 'map_medium': tensor(0.0906),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.0688),
 'mar_1': tensor(0.0551),
 'mar_10': tensor(0.0735),
 'mar_100': tensor(0.0735),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.0457),
 'mar_medium': tensor(0.1003),
 'mar_small': tensor(0.0872)}
key2 -  1-2
{'map': tensor(0.1196),
 'map_50': tensor(0.2320),
 'map_75': tensor(0.0973),
 'map_large': tensor(0.1289),
 'map_medium': tensor(0.1304),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.0747),
 'mar_1': tensor(0.0952),
 'mar_10': tensor(0.1385),
 'mar_100': tensor(0.1385),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.1429),
 'mar_medium': tensor(0.1509),
 'mar_small': tensor(0.0957)}
key2 -  1-4
{'map': tensor(0.1532),
 'map_50': tensor(0.2946),
 'map_75': tensor(0.1237),
 'map_large': tensor(0.1554),
 'map_medium': tensor(0.1374),
 'map_per_c

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
original = []
without_noise = []
for key in preds.keys():
    print(key)
    without_noise.append(
        {
            "boxes": torch.tensor(preds[key]['bboxes']),
            "labels": torch.tensor(preds[key]['labels']),
            "scores": torch.tensor(preds[key]['scores'])
        }.copy()
    )

    original.append(
        {
            "boxes": torch.tensor(preds[key]['original']['bboxes']),
            "labels": torch.tensor(preds[key]['original']['labels'])
        }.copy()
    )

print("without_noise - ", without_noise)
print("original - ", original)
metric = MeanAveragePrecision()
metric.update(without_noise, original)
pprint(metric.compute())           